In [2]:
import numpy as np
from PIL import Image
from ImageFrame import Frame
from EdgeFinderV4 import BoundaryDetector as BD2
import timeit
import cv2 as cv
import matplotlib.pylab as plt
from concurrent.futures import ThreadPoolExecutor

In [3]:
img = np.array(Image.open("TestingImage/Image3.jpg"))
img1 =Image.open("TestingImage/Image3.jpg")
frame1 = Frame(1280,720,10)

load CVS file successfully


In [3]:
kernal = np.array([-1,-1,0,1,1])
steps = 0
T_hold = 21
point =np.empty([0])
def sim_many_img(i):
    img = np.array(Image.open("TestingImage/Image"+str(i)+".jpg"))
    t1=timeit.default_timer()
    _ = BD2(img,kernal,frame1,steps,T_hold)
    t2=timeit.default_timer()
    print(f'runtime image#{i} = {t2-t1}')

In [4]:
%%time
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(8) as ex:
    ex.map(sim_many_img,range(100))

runtime image#6 = 3.9762019000045257runtime image#2 = 4.051682400000573
runtime image#4 = 3.9858041999978013
runtime image#3 = 4.03011969999352
runtime image#1 = 4.056360200003837
runtime image#5 = 3.936892000005173

runtime image#7 = 3.9225685999990674
runtime image#0 = 4.08849210000335
runtime image#8 = 0.007289699999091681
runtime image#11 = 0.011739699999452569
runtime image#13 = 0.031196099997032434
runtime image#10 = 0.03025569999590516
runtime image#9 = 0.004260000001522712
runtime image#12 = 0.012134299999161158
runtime image#14 = 0.0037603000018862076
runtime image#15 = 0.04078420000587357
runtime image#17 = 0.005838000004587229
runtime image#23 = 0.004426199993758928
runtime image#20 = 0.003913799999281764
runtime image#22 = 0.020746000001963694
runtime image#16 = 0.04391290000057779
runtime image#21 = 0.006836299995484296
runtime image#18 = 0.01531039999827044
runtime image#19 = 0.023112999995646533
runtime image#24 = 0.0015438000045833178
runtime image#29 = 0.01500720000331

In [9]:
m1_time = np.empty([0,0])
m2_time = np.empty([0,0])
m3_time = np.empty([0,0])

kernal = np.array([-1,-1,0,1,1])
steps = 0
T_hold = 75
ddepth = cv.CV_16S

for i in range(50):
    img = np.array(Image.open("TestingImage/Image"+str(i)+".jpg"))
    img1 = cv.imread("TestingImage/Image"+str(i)+".jpg")
  

    # Pre-process for Sobel algorithm

    t1 = timeit.default_timer()
    # Modified Edge detection algorithm
    
    B = BD2(img,kernal,frame1,steps,T_hold) 

    t2 = timeit.default_timer()
    # Canny operator
    c=cv.Canny(img1,100,200)
    
    t3 = timeit.default_timer()

    # Sobel Operator
    img3 = cv.GaussianBlur(img1,(3,3),0)
    img2 = cv.cvtColor(img3,cv.COLOR_BGR2GRAY)
    grad_x = cv.Sobel(img2, ddepth, 1, 0, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)
    grad_y = cv.Sobel(img2, ddepth, 0, 1, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_y = cv.convertScaleAbs(grad_y)
    grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    t4 = timeit.default_timer()

    
    #B.ploting_onImage(img,str(i)+" \n our_method:"+str(np.round(t2-t1,4))+"s\n Canny:"+str(np.round(t3-t2,4))+"s")
    #plt.show()


    m1_time = np.append(m1_time,t2-t1)
    m2_time = np.append(m2_time,t3-t2)
    m3_time = np.append(m3_time,t4-t3)


#printing statistic for both method
print(f'Our_method : max={np.max(m1_time): .4f} min={np.min(m1_time): .4f} average={np.average(m1_time): .4f}')
print(f'Canny_method : max={np.max(m2_time): .4f} min={np.min(m2_time): .4f} average={np.average(m2_time): .4f}')
print(f'Sobel_method : max={np.max(m3_time): .4f} min={np.min(m3_time): .4f} average={np.average(m3_time): .4f}')

Our_method : max= 0.0039 min= 0.0015 average= 0.0022
Canny_method : max= 0.0178 min= 0.0085 average= 0.0110
Sobel_method : max= 0.0097 min= 0.0051 average= 0.0066
